In [55]:

import warnings
import sys
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
import sys

In [56]:
tech_tickers = ["AAPL","AMZN","TSLA", "NVDA", "MSFT", "GOOG","INTC", "DELL"]

In [57]:

class Returns_analysis():
    individual_stock_data = pd.DataFrame()
    closing_stock_data = pd.DataFrame()
    
   
    def __init__(self,tickers : list,start_date):
            self.tickers = tickers
            self.start_date = start_date
           
    
    def individual_data(self,stock_dict,fill_method : str):
        yf.pdr_override()
        valid_fill_methods = ['linear', 'forward', 'backward']
        if fill_method not in valid_fill_methods:
                raise ValueError(f"Invalid fill method. Choose one of {valid_fill_methods}")
        if type(stock_dict) != dict:
                raise TypeError(f"Data type should be of type dictionary")
        for ticks in self.tickers:
                Returns_analysis.individual_stock_data = pdr.DataReader(ticks, self.start_date)
                stock_dict.update({f"{ticks}_stock" : Returns_analysis.individual_stock_data})
                if fill_method == "forward":
                     Returns_analysis.individual_stock_data.fillna(method = "ffill",inplace = True)
                elif fill_method == "backward":
                     Returns_analysis.individual_stock_data.fillna(method = "bfill", inplace = True)
                else:
                     Returns_analysis.individual_stock_data.interpolate(method=fill_method, inplace=True) # linear fill missing values
                stock_dict.update({f"{ticks}_stock" : Returns_analysis.individual_stock_data})


    def download_stocks_closing(self,fill_method, column_name: str):
          yf.pdr_override()
          valid_fill_methods = ['linear', 'forward', 'backward']
          if fill_method not in valid_fill_methods:
                raise ValueError(f"Invalid fill method. Choose one of {valid_fill_methods}")
          self.column_name = column_name
          for ticks in self.tickers:
                Returns_analysis.closing_stock_data[ticks] = pdr.DataReader(ticks, self.start_date)[column_name]
                if fill_method == "forward":
                   Returns_analysis.closing_stock_data[ticks].fillna(method="ffill", inplace=True)
                elif fill_method == "backward":
                   Returns_analysis.closing_stock_data[ticks].fillna(method="bfill", inplace=True)
                else:
                   Returns_analysis.closing_stock_data[ticks].interpolate(method=fill_method, inplace=True)
          return Returns_analysis.closing_stock_data




    def get_returns(self,return_type):
           self.return_type = return_type
           if return_type == "Log":
                returns_data = np.log(Returns_analysis.closing_stock_data/Returns_analysis.closing_stock_data.shift(1))
           else: 
                returns_data = (Returns_analysis.closing_stock_data/Returns_analysis.closing_stock_data.shift(1))-1
           return returns_data

    def normalization(self):
            normalized_data = (Returns_analysis.closing_stock_data/Returns_analysis.closing_stock_data.iloc[0]*100)
            normalized_figure = normalized_data.plot(figsize = (19,6))
            plt.title("Growth Comparison of Stocks")
            return normalized_figure
    
    def individual_return(self):
            individual_return = self.get_returns(self.return_type).mean()*250*100

            print(f"The {self.return_type} return of the stocks are {round(individual_return,2)}")
        

    def portfolio_return(self,weights = list):
            import sys
            weights = np.array(weights)
            self.weights = weights
            if round(sum(weights),2)!= 1 :
                raise ValueError(f"Sum of the weight of the porfolio should be 1,The sum of your weights are {sum(weights)}")
                if len(weights) != Returns_Analysis.closing_stock_data.shape[1]:
                    sys.exit("The weights of individual security does not match the no. of securities")
            annual_returns = np.dot(self.get_returns(self.return_type).mean()*250, weights)

            print(f"The annual returns of the portfolio is {round(annual_returns, 2)}% ")


    def individual_risk_factor(self):
                risk_factor = pd.DataFrame(np.sqrt(self.get_returns(self.return_type).var()*250))
                print(risk_factor)

                print(f"The lowest risk factor is{risk_factor.min()}")

    def portfolio_variance(self):
        if self.weights is None:
            raise ValueError("Weights are not specified")
        returns = self.get_returns(self.return_type)
        cov_matrix = np.cov(returns.T)
        portfolio_variance = np.dot(self.weights.T, np.dot(cov_matrix, self.weights))
        print(f"The portfolio variance is {round(portfolio_variance, 2)}")

In [58]:
securities = Returns_analysis(tech_tickers, start_date = "2010-1-1")

In [59]:
securities.download_stocks_closing(column_name = "Adj Close", fill_method = "forward")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AAPL,AMZN,TSLA,NVDA,MSFT,GOOG,INTC,DELL
Date,,,,,,,,
2010-01-04,6.515213,6.695000,NaN,4.242310,23.683071,15.610239,14.020262,NaN
2010-01-05,6.526477,6.734500,NaN,4.304259,23.690722,15.541497,14.013548,NaN
2010-01-06,6.422664,6.612500,NaN,4.331791,23.545336,15.149715,13.966545,NaN
2010-01-07,6.410790,6.500000,NaN,4.246899,23.300468,14.797037,13.832250,NaN
2010-01-08,6.453413,6.676000,NaN,4.256077,23.461161,14.994298,13.986688,NaN
...,...,...,...,...,...,...,...,...
2023-01-31,144.289993,103.129997,173.220001,195.369995,247.809998,99.870003,28.260000,40.619999
2023-02-01,145.429993,105.150002,181.410004,209.429993,252.750000,101.430000,29.070000,41.230000
2023-02-02,150.820007,112.910004,188.270004,217.089996,264.600006,108.800003,30.190001,42.419998


In [60]:
dicts = {}
securities.individual_data(stock_dict= dicts, fill_method= "forward")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [61]:
dicts["AMZN_stock"]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,6.812500,6.830500,6.657000,6.695000,6.695000,151998000
2010-01-05,6.671500,6.774000,6.590500,6.734500,6.734500,177038000
2010-01-06,6.730000,6.736500,6.582500,6.612500,6.612500,143576000
2010-01-07,6.600500,6.616000,6.440000,6.500000,6.500000,220604000
2010-01-08,6.528000,6.684000,6.451500,6.676000,6.676000,196610000
...,...,...,...,...,...,...
2023-01-31,101.160004,103.349998,101.139999,103.129997,103.129997,66527300
2023-02-01,102.529999,106.239998,101.239998,105.150002,105.150002,80450100
2023-02-02,110.250000,114.000000,108.879997,112.910004,112.910004,158154200
